# Query non-stem data from mongoDB

In [ ]:
import pymongo
import urllib
from pymongo import MongoClient
from datetime import date
import datetime

import pandas as pd
from pandas import Series, DataFrame
import numpy as np

import time
import timeit

In [ ]:
username = urllib.parse.quote_plus('#######')
password = urllib.parse.quote_plus('#######')
clinet = MongoClient('mongodb://%s:%s@###########' % (username, password))

In [ ]:
db=clinet['Production']
collection=db['ParsedJobads']

In [ ]:
#query specific all wanted data
start = time.time()
fill= {'description':1, 'title_en':1, 'title_th':1, 'mine_result.Occupation.EN':1,
       'mine_result.industry.group':1, 'company':1 }
data= list(collection.find({},fill))
end = time.time()
print('runtime = ', end-start)

runtime =  298.4246470928192


In [ ]:
len(data)

1082318

In [ ]:
dataF=pd.DataFrame(data)
len(dataF)

1082318

In [ ]:
non_stem=dataF[dataF['mine_result']=={'Occupation': {'EN': 'non STEM Occupation'}}]
len(non_stem)

721174

In [ ]:
non_stem.head()

,_id,title_en,title_th,description,company,mine_result
0,5cfe0339ae04a37efca33a0f,,Shop Manager - Luxury Brand,OverviewWe are looking for an experiencedShop ...,"Qapstone Consulting Group Recruitment Co., Ltd.",{'Occupation': {'EN': 'non STEM Occupation'}}
1,5cfe0339ae04a37efca33a0d,,Accounting Manager,Accounting ManagerJob descriptionManage balanc...,"THE MAKE-IT CO., LTD.",{'Occupation': {'EN': 'non STEM Occupation'}}
2,5cfe0339ae04a37efca33a0e,,HR Officer / เจ้าหน้าที่ทรัพยากรมนุษย์,รายละเอียดงาน:ควบคุมอัตรากำลัง งานสรรหาและว่าจ...,"Sri Trang Rubber & Plantation Co., Ltd.",{'Occupation': {'EN': 'non STEM Occupation'}}
3,5cfe0339ae04a37efca33a10,,พนักงานสินค้า/ส่งเอกสารประจำ อำเภอเมือง จังหวั...,Responsibilitiesส่งเอกสารที่ได้รับหมอบหมายให้ส...,"U Delivery Co., Ltd.",{'Occupation': {'EN': 'non STEM Occupation'}}
4,5cfe0339ae04a37efca33a0c,,Overseas Buyer/จัดซื้อ(ต่างประเทศ),Job main duties:Prepare costing and control ma...,"Sybarite Co., Ltd.",{'Occupation': {'EN': 'non STEM Occupation'}}


In [ ]:
temp=[]
for i in non_stem['description']:
  if len(str(i))>150:
    temp.append(i)
len(temp)

551990

In [ ]:
select=pd.Series(temp)
select.head()

0    OverviewWe are looking for an experiencedShop ...
1    Accounting ManagerJob descriptionManage balanc...
2    รายละเอียดงาน:ควบคุมอัตรากำลัง งานสรรหาและว่าจ...
3    Responsibilitiesส่งเอกสารที่ได้รับหมอบหมายให้ส...
4    Job main duties:Prepare costing and control ma...
dtype: object

In [ ]:
len(select)

551990

In [ ]:
select[1]

'Accounting ManagerJob descriptionManage balance sheetAccurate month, quarter and year end reportingCoordinate with external auditorAnalyze financial resultsProduction costingExcellent knowledge of particularities of BOI accountingProfile:Bachelor degree or higher in Accounting or FinanceMinimal 5 years working experience in accounting or finance managerial positionConfident to communicate in EnglishHands on mentality, capable of setting up and running a department'

In [ ]:
# for testing purpose
select=select[:100000]

In [ ]:
len(select)

100000

# preprocess English and Thai texts 

In [ ]:
#import ENG
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#THAI
!pip install pythainlp

     |████████████████████████████████| 10.6MB 4.3MB/s 
     |████████████████████████████████| 747kB 40.2MB/s 


In [ ]:
!pip install epitran

     |████████████████████████████████| 143kB 4.2MB/s eta 0:00:01
     |████████████████████████████████| 276kB 7.5MB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp37-none-any.whl size=10768 sha256=957fcd5883628f5e8ad0fba27423e5d3a1acadcaa27d02b746d9d9a900e638d6
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp37-cp37m-linux_x86_64.whl size=861704 sha256=3fcc21f5e58ec87f674a0db2eb924700c7ff4d9439d5888d5701f05039ec89d9
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built unicodecsv marisa-trie


In [ ]:
import pythainlp
from pythainlp import word_tokenize

pythainlp.__version__

'2.2.6'

In [ ]:
# all function

#function to solve: ManagerFor, managerFor/ out = split words
#input = a word (only English)
#output = [w1, w2, w3,...]
import re
def splitword(text):
  out=[]
  couUp=0
  left=0
  initial=text.strip()
  upper=re.findall('[A-Z]',initial)
  for i in range(len(initial)):
    if initial[i] in upper:
      couUp+=1
    if couUp==len(upper):
      out.append(initial[left:i])
      out.append(initial[i:])
      break
    elif initial[i] in upper and i !=0:
      out.append(initial[left:i])
      left=i
  return ' '.join(out).strip()

#Data Pre-processing for English only
# input = many words
#output = [w1, w2, w3, ......]

#1. Tokenization: Split the text into sentences and the sentences into words.
# Lowercase the words and remove punctuation.
#2. Words that have fewer than 3 characters are removed.
#3. All stopwords are removed.
#4. Words are lemmatized — 
#words in third person are changed to first person and verbs in past and future tenses are changed into present.
#5. Words are stemmed — words are reduced to their root form.
def lemmatize_stemming(text):
    return SnowballStemmer('english').stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

#process for all data
#input = docs
# output = a list of each word
def preprocessAll(text):
  text=str(text)
  text=text.strip()
  thai=pythainlp.thai_characters
  th=''
  en=''
  for i in range(len(text)):
    if text[i] in thai:
      th=th+text[i]
    else:
      if (i != 0) and (text[i-1] in thai):
        en=en+' '+text[i]
      else:
        en=en+text[i]
  temp_en=[]
  for i in en.split(' '):
    if i !='':
      temp_en.append(splitword(i))
  temp_en=' '.join(temp_en)
  return preprocess(temp_en)+word_tokenize(th, keep_whitespace=False)

In [ ]:
#map processAll with the data
start = timeit.default_timer()
processed_docs = select.map(preprocessAll)
stop = timeit.default_timer()
print('runime: ', stop - start)  

runime:  241.39133082800004


In [ ]:
#all eng
processed_docs[0]

['overview',
 'look',
 'experi',
 'shop',
 'managerfor',
 'client',
 'know',
 'luxuri',
 'retail',
 'role',
 'organ',
 'respons',
 'aspect',
 'supervis',
 'shop',
 'includ',
 'sale',
 'staff',
 'stock',
 'resourc',
 'manag',
 'respons',
 'ensur',
 'staff',
 'great',
 'custom',
 'servic',
 'custom',
 'monitor',
 'sale',
 'perform',
 'store',
 'maintain',
 'statist',
 'financi',
 'record',
 'deal',
 'custom',
 'queri',
 'complaint',
 'overse',
 'price',
 'stock',
 'control',
 'manag',
 'overal',
 'aspect',
 'store',
 'manag',
 'staff',
 'peopl',
 'qualif',
 'femal',
 'year',
 'bachelor',
 'degre',
 'equival',
 'field',
 'good',
 'command',
 'english',
 'languag',
 'advantag',
 'prior',
 'experi',
 'work',
 'retail',
 'store',
 'experi',
 'manag',
 'staff',
 'high',
 'prefer',
 'previous',
 'experi',
 'manag',
 'team',
 'abl',
 'work',
 'day',
 'week',
 'benefit',
 'posit',
 'allow',
 'commiss',
 'month',
 'day',
 'month',
 'health',
 'life',
 'insur',
 'provid',
 'fund',
 'inquiri',
 'co

In [ ]:
#all thai
processed_docs[2]

['รายละเอียด',
 'งาน',
 'ควบคุม',
 'อัตรา',
 'กำลังงาน',
 'สรรหา',
 'และ',
 'ว่าจ้าง',
 'พนักงาน',
 'รายวัน',
 'งาน',
 'แรงงานสัมพันธ์',
 'งานธุรการ',
 'คุณสมบัติ',
 'ชาย',
 'อายุ',
 'ปี',
 'ขึ้นไป',
 'วุฒิการศึกษา',
 'ปริญญาตรี',
 'การบริหาร',
 'ทรัพยากร',
 'มนุษย์',
 'หรือ',
 'สาขา',
 'ที่',
 'เกี่ยวข้อง',
 'สามารถ',
 'เดินทาง',
 'ไป',
 'ปฏิบัติงาน',
 'ต่างจังหวัด',
 'ได้',
 'มีประสบการณ์',
 'ใน',
 'การ',
 'สรรหา',
 'พนักงาน',
 'จะ',
 'พิจารณา',
 'เป็นพิเศษ',
 'มี',
 'มนุษยสัมพันธ์',
 'ดี',
 'สามารถ',
 'ทำงาน',
 'ภายใต้',
 'ภาวะ',
 'กดดัน',
 'ได้ดี',
 'สวัสดิการ',
 'ประกันสังคม',
 'โบนัส',
 'กองทุน',
 'น',
 'สำรอง',
 'เลี้ยงชีพ',
 'ประกัน',
 'อุบัติเหตุ',
 'ประกัน',
 'สุขภาพ',
 'ติดต่อ',
 'คุณ',
 'หนึ่ง',
 'ฤทัย',
 'กันทะ',
 'ยอง',
 'บริษัท',
 'ศรีตรัง',
 'รับ',
 'เบอร์',
 'แอนด์',
 'แพลน',
 'เท',
 'ชั่น',
 'จำกัด',
 'มต',
 'หนอง',
 'ป่า',
 'ครั่ง',
 'อ',
 'เมือง',
 'จ',
 'เชียงใหม่',
 'จังหวัด',
 'เชียงใหม่',
 'โทร']

In [ ]:
#eng, thai mixed
processed_docs[3]

['qualif',
 'benefit',
 'contact',
 'prontima',
 'ส่ง',
 'เอกสาร',
 'ที่',
 'ได้รับ',
 'หมอบ',
 'หมาย',
 'ให้',
 'สำเร็จ',
 'ส่ง',
 'เอกสาร',
 'ให้',
 'ถึง',
 'มือ',
 'ลูกค้า',
 'โดย',
 'ถูกต้อง',
 'เพศชาย',
 'อายุ',
 'ปี',
 'วุฒิการศึกษา',
 'ม',
 'ปริญญาตรี',
 'มี',
 'ที่อยู่อาศัย',
 'ใน',
 'อำเภอ',
 'เมือง',
 'จังหวัด',
 'สกลนคร',
 'สามารถ',
 'ปฏิบัติ',
 'งานประจำ',
 'ที่',
 'ศูนย์',
 'กิฟฟารีน',
 'จ',
 'สกลนคร',
 'ได้',
 'มี',
 'ใบ',
 'อนุ',
 'ญาติ',
 'ขับขี่',
 'รถจักรยานยนต์',
 'มี',
 'รถจักรยานยนต์',
 'เป็น',
 'ของ',
 'ตัวเอง',
 'มี',
 'ความ',
 'ขยัน',
 'อดทน',
 'ซื่อสัตย์',
 'ต่อ',
 'องค์กร',
 'เริ่ม',
 'งาน',
 'ได้',
 'ทันที',
 'จะ',
 'พ',
 'จาร',
 'ณา',
 'เป็นพิเศษ',
 'ทำงาน',
 'จส',
 'เวลา',
 'มี',
 'เบี้ย',
 'ขยัน',
 'และ',
 'สวัสดิ',
 'การตาม',
 'ประกันสังคม',
 'มี',
 'ความมั่นคง',
 'ใน',
 'ตำแหน่งหน้าที่',
 'ซอย',
 'รามคำแหง',
 'ใกล้',
 'เซเว่น',
 'กลาง',
 'ซอย',
 'ถนน',
 'รามคำแหง',
 'แขวง',
 'พลับพลา',
 'เขต',
 'วังทองหลาง',
 'กรุงเทพมหานคร']

In [ ]:
#create maping btw a word and an integer id
start = timeit.default_timer()
dictionary = gensim.corpora.Dictionary(processed_docs)
stop = timeit.default_timer()
print('runime: ', stop - start)  

runime:  10.263946713999985


In [ ]:
dictionary[0]

'abl'

In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abl
1 advantag
2 allow
3 aspect
4 bachelor
5 benefit
6 client
7 command
8 commiss
9 complaint
10 contact


In [ ]:
#fillter words that appear less than no_bow docs + more than no_above (fraction)
# and keep the first most frequent words = keep_n

#keep_n = English + Thai so 2*100000 = 200000 ?

start = timeit.default_timer()
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
stop = timeit.default_timer()
print('runime: ', stop - start)  

runime:  0.11493839100000969


In [ ]:
#doc2bow = list of (id of a word, no_appear in a doc) 
start = timeit.default_timer()
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
stop = timeit.default_timer()
print('runime: ', stop - start)  

runime:  7.962516048999987


In [ ]:
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 3),
 (13, 2),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 4),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 5),
 (39, 1),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 2),
 (58, 2),
 (59, 1),
 (60, 2),
 (61, 1),
 (62, 2),
 (63, 4),
 (64, 1),
 (65, 2),
 (66, 3),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 2),
 (71, 1)]

In [ ]:
bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

Word 0 ("abl") appears 1 time.
Word 1 ("advantag") appears 1 time.
Word 2 ("allow") appears 1 time.
Word 3 ("aspect") appears 2 time.
Word 4 ("bachelor") appears 1 time.
Word 5 ("benefit") appears 1 time.
Word 6 ("client") appears 1 time.
Word 7 ("command") appears 1 time.
Word 8 ("commiss") appears 1 time.
Word 9 ("complaint") appears 1 time.
Word 10 ("contact") appears 1 time.
Word 11 ("control") appears 1 time.
Word 12 ("custom") appears 3 time.
Word 13 ("day") appears 2 time.
Word 14 ("deal") appears 1 time.
Word 15 ("degre") appears 1 time.
Word 16 ("english") appears 1 time.
Word 17 ("ensur") appears 1 time.
Word 18 ("equival") appears 1 time.
Word 19 ("experi") appears 4 time.
Word 20 ("femal") appears 1 time.
Word 21 ("field") appears 1 time.
Word 22 ("financi") appears 1 time.
Word 23 ("fund") appears 1 time.
Word 24 ("good") appears 1 time.
Word 25 ("great") appears 1 time.
Word 26 ("health") appears 1 time.
Word 27 ("high") appears 1 time.
Word 28 ("includ") appears 1 time.


In [ ]:
len(bow_corpus)

100000

# modelling

In [ ]:
#model for calculate weight(related to fraction of docs that a word occur vs a word frequency in a doc)
#extract main words that represent the doc 
# weight = (a word frequency in a doc) + inv(doc frequency)
# ex assume topic = [salad, pizza], f(food) high in doc frequency (food occur in all topics)
# but vegetable occur most only in salad so f(vegetable) only high in a word frequency but low in doc frequency 
start = timeit.default_timer()
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
stop = timeit.default_timer()
print('runime: ', stop - start)  

runime:  1.4007419590000154


In [ ]:
from pprint import pprint
cou=0
for doc in corpus_tfidf:
    pprint(doc)
    cou+=1
    if cou>=5:
      break

[(0, 0.0660343652403194),
 (1, 0.0792960511720394),
 (2, 0.10320772597896419),
 (3, 0.22422907547479434),
 (4, 0.04870830205531922),
 (5, 0.07277731243740862),
 (6, 0.06903867115236706),
 (7, 0.06525808805379475),
 (8, 0.1134322857469969),
 (9, 0.11277515302759963),
 (10, 0.06199174506644908),
 (11, 0.0687047005806943),
 (12, 0.15911748725350858),
 (13, 0.22274833887482265),
 (14, 0.09923052270624183),
 (15, 0.0466129561938495),
 (16, 0.045260624514608605),
 (17, 0.05522490116900568),
 (18, 0.12093210769105685),
 (19, 0.16721159477587957),
 (20, 0.07489919482219673),
 (21, 0.054957063116067496),
 (22, 0.07819253187545576),
 (23, 0.09779421802316507),
 (24, 0.04740299586592918),
 (25, 0.1144504060957578),
 (26, 0.09782831695997106),
 (27, 0.07215473735235414),
 (28, 0.06300377166999435),
 (29, 0.12169378000408942),
 (30, 0.08767792263472515),
 (31, 0.12218512564839135),
 (32, 0.09136151506745026),
 (33, 0.10511069100433586),
 (34, 0.07431550458158145),
 (35, 0.09003234661942383),
 (36, 

In [ ]:
#run model using corpus_tfidf

#num_topic = 20 ?

start = timeit.default_timer()
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, workers=None)
stop = timeit.default_timer()
print('runime: ', stop - start)  

runime:  106.42191962699997


In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"ลูกค้า" + 0.010*"การ" + 0.008*"ใน" + 0.008*"ของ" + 0.008*"ให้" + 0.008*"ขาย" + 0.007*"กับ" + 0.006*"เพื่อ" + 0.006*"สินค้า" + 0.006*"การตลาด"
Topic: 1 Word: 0.009*"เซ็นทรัล" + 0.008*"ภูเก็ต" + 0.007*"จตุจักร" + 0.007*"ร้าน" + 0.006*"ลูกค้า" + 0.006*"รัชดาภิเษก" + 0.006*"เอ๋" + 0.006*"การบริหารธุรกิจ" + 0.005*"ซ่อม" + 0.005*"ชงกาแฟ"
Topic: 2 Word: 0.015*"นนท์" + 0.013*"south" + 0.012*"กุ๊ก" + 0.012*"yannawa" + 0.011*"รัชดาภิเษก" + 0.011*"sathorn" + 0.011*"ตำบล" + 0.010*"อรุณ" + 0.009*"ครัว" + 0.008*"phayathai"
Topic: 3 Word: 0.012*"commerci" + 0.010*"domest" + 0.009*"bldg" + 0.008*"verbal" + 0.008*"segment" + 0.007*"adapt" + 0.007*"trad" + 0.007*"user" + 0.007*"ปฐมวัย" + 0.007*"centr"
Topic: 4 Word: 0.022*"สินค้า" + 0.012*"ยอดขาย" + 0.011*"สต็อก" + 0.010*"ให้" + 0.009*"คีย์" + 0.009*"ลง" + 0.009*"ครบถ้วน" + 0.009*"แนะนำ" + 0.009*"เช็ค" + 0.009*"ค้าปลีก"
Topic: 5 Word: 0.026*"วัน" + 0.018*"บาท" + 0.017*"ทำงาน" + 0.016*"ชั่วโมง" + 0.013*"ศุกร์" + 0.013*"จันทร์" + 0.0

# use the model

In [ ]:
def predict_tfid(text):
  process_unseen= preprocessAll(text)
  dic_unseen=gensim.corpora.Dictionary([process_unseen])
  bow_unseen=dic_unseen.doc2bow(process_unseen)
  target=bow_unseen
  return sorted(lda_model_tfidf[target], key= lambda x:x[1], reverse= True)

In [ ]:
predict_tfid('ดูแลวางแผนการผลิตในแต่ละหน่วยย่อยของการผลิต ติดตาม แก้ปัญหา และควบคุมการปฏิบัติงานให้เสร็จตามกำหนด')

[(11, 0.95)]

# save the model

In [ ]:
import joblib
model=lda_model_tfidf
filename = 'non_stem_v1_model.sav'
joblib.dump(model, filename)

['non_stem_v1_model.sav']

# load the model

In [ ]:
loaded_model = joblib.load(filename)

In [ ]:
loaded_model

In [ ]:
text='ดูแลวางแผนการผลิตในแต่ละหน่วยย่อยของการผลิต ติดตาม แก้ปัญหา และควบคุมการปฏิบัติงานให้เสร็จตามกำหนด'
process_unseen= preprocessAll(text)
dic_unseen=gensim.corpora.Dictionary([process_unseen])
bow_unseen=dic_unseen.doc2bow(process_unseen)
target=bow_unseen
sorted(loaded_model[target], key= lambda x:x[1], reverse= True)

[(11, 0.9499999)]